# Hand Keypoint Detection using YOLOv11

## Introduction

This notebook will guide you through using YOLOv11 (You Only Look Once) Algorithm to detect 21 different keypoints on human hand by training the YOLO model on a Ultralytics Hand Keypoints dataset which contains over 26,768 images.

## Credits

- Dataset: [Roboflow](https://docs.ultralytics.com/datasets/pose/hand-keypoints/)
- Reference: [Roboflow](https://docs.ultralytics.com/datasets/pose/hand-keypoints/)
- Model: [Ultralytics](https://github.com/ultralytics/ultralytics)

## Table of Contents

- [Introduction](#introduction)
- [Prerequisites](#prerequisites)
- [Installing YOLOv11](#installing-yolov11)
- [Getting the Dataset](#getting-the-dataset)
- [Training](#training)
- [Testing](#testing)
- [Performance](#performance)  
- [WebCam Testing](#webcam-testing)
- [Usage](#usage)

## Prerequisites

We'll be using following tools for this notebook
- Ultralytics YOLOv11 Model
- Roboflow Car parts Dataset

Make sure you have access to GPU for faster computation. Run `nvidia-smi` command and check if you get output something like following

In [ ]:
!nvidia-smi

Check if the current working directory is root directory of project

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Installing YOLOv11

Ultralytics package includes all necessary libraries and dependencies used to run YOLOv11. So installation is quite simple

In [ ]:
!pip install ultralytics

Make sure that ultralytics is installed correctly

In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.9 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (Tesla V100-PCIE-32GB, 32510MiB)
Setup complete ✅ (64 CPUs, 376.6 GB RAM, 120.9/136.5 GB disk)


## Getting the Dataset

Dataset will be fetched using the predefined yaml file for hand keypoints dataset. The training will start after fetching the dataset.

- Name: Ultralytics Hand Keypoints Dataset
- Format: YOLOv11
- Images count: 26,768
- Image size: 640x640
- Keypoints count: 21
- Train/Val/Test Distribution: 70%:30%:0%

## Training

Run the following code to get the dataset and start training.

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="hand-keypoints/data.yaml", epochs=10, imgsz=640, save=True, device=0)

**NOTE**: Some images in the dataset might be corrupted or non-normalized. You can ignore the warnings related to them if you encounter any.

The model was trained for only 10 epochs because of resource constraints, since long training times cause GPU overheating and throttling and cause the process to slow down.  You can increase the number of epochs for better results if you have more resources available.

## Testing

Let's test our model performance on the validation set.

In [ ]:
model = YOLO(f"{HOME}/runs/pose/train/weights/best.pt") 

metrics = model.val() # No need to explicitly pass arguments: imgsz, data, conf, batch, etc. model will use the values specified during training
metrics.box.map  
metrics.box.map50  
metrics.box.map75 
metrics.box.maps 

## Performance

YOLO provides a detailed visualisation of the model performance. Check out `runs/pose/train` folder for training performance, and `runs/pose/val` folder for validation performance.

We achieved a mAP of 0.805 which is sufficiently reliable for any keypoint detection task.

## WebCam Testing

You can detect hand keypoints in realtime using OpenCV library for performing detections of Video Stream from Webcam

In [9]:
import cv2
from ultralytics import YOLO

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

model = YOLO('best.pt')

while True:

    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame.")
        break

    result = model.predict(frame)

    cv2.imshow('Webcam Video Stream', result[0].plot())

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<Figure size 1200x1200 with 1 Axes>

## Usage

YOLO weights are now upgraded to detect hand keypoints. Now you can use the trained weights for detecting hand gestures.

The trained weights are stored in `runs/pose/train/weights/best.pt` file. To use the trained weights on you images, first load the weights in a YOLO model. Then, use the model to detect hand keypoints in your images.

## Portable Weights

Now you can use you trained model any any machine with ultralytics installed. Just copy the weights `runs/pose/train/weights/best.pt` to your machine and load the YOLO model with those weights.

Your model is now ready to _read_ your gestures!